In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os

path="drive/MyDrive/tn_lab2/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
from pyswip import Prolog
import json

In [ ]:
data = pd.read_csv(path + "movies_metadata.csv")

data.fillna("UNK", inplace=True)

In [ ]:
iterator = 0
for i,row in enumerate(data['production_companies']):
  try:
    studios = data['production_companies'][i]
    studios = json.loads(studios.replace(u"'", u'"'))
  except:
    iterator += 1
    #print(f"{i} has an issue: {data['production_companies'][i]}")
print(iterator)

71


In [ ]:
def clean_text(text):
  text = text.replace(u'\xa0', u'')
  text = text.replace(u"'", u'')
  return text

## Creating Literals and Assertions

In [ ]:
prolog = Prolog()

literals = []
movie_score = []
for i, row in enumerate(data.itertuples(index=True, name='Pandas')):
  movie_title = clean_text(getattr(row, 'movie_title'))
  keywords = clean_text(getattr(row, 'plot_keywords'))
  language = clean_text(getattr(row, 'language'))
  director = clean_text(getattr(row, 'director_name'))
  actor1 = clean_text(getattr(row, 'actor_1_name'))
  actor2 = clean_text(getattr(row, 'actor_2_name'))
  actor3 = clean_text(getattr(row, 'actor_3_name'))
  country = clean_text(getattr(row, 'country'))
  year = str(getattr(row, "release_date").split('-')[0])
  try:
    studios = data['production_companies'][i]
    studios = json.loads(studios.replace(u"'", u'"'))
  except:
    continue
  score = clean_text(str(getattr(row, "vote_average")))
  decade = f'{year[2]}0"s'



  if 'black and white' in keywords:
    literals.append("black_and_white('"+ movie_title +"','"+ 'yes' +"')")
  else:
    literals.append("black_and_white('"+ movie_title +"','"+ 'no' +"')")

  literals.append("year('"+ movie_title +"','"+ year +"')")
  literals.append("language('"+ movie_title +"','"+ language +"')")
  literals.append("director('"+ movie_title +"','"+ director +"')")
  literals.append("actor1('"+ movie_title +"','"+ actor1 +"')")
  literals.append("actor2('"+ movie_title +"','"+ actor2 +"')")
  literals.append("actor3('"+ movie_title +"','"+ actor3 +"')")
  literals.append("decade('"+ movie_title +"','"+ decade +"')")
  literals.append("score('"+ movie_title +"','"+ score +"')")
  literals.append("country('"+ movie_title +"','"+ country +"')")

  for studio in studios:
    literals.append("studio('"+ movie_title +"','"+ studio['name'] +"')")

  for keyword in keywords.split("|"):
    literals.append("keyword('"+ movie_title +"','"+ keyword +"')")

  for genre in getattr(row, 'genres').split("|"):
    literals.append("genre('"+ movie_title +"','"+ genre +"')")

literals.sort()
for literal in literals:
  prolog.assertz(literal)

prolog.consult(path + "db.pl")

## Implementing some Analysis

In [ ]:
data['vote_average']

0       7.2
1       6.9
2       6.3
3       7.6
4       6.1
       ... 
4798    6.6
4799    5.9
4800    7.0
4801    5.7
4802    6.3
Name: vote_average, Length: 4803, dtype: float64

In [ ]:
import pandas as pd
data_filou = pd.read_csv(path + 'train_ratings.csv')
favorite_movies = list(data_filou[data_filou['rating'] > 3]['movie_title'].values)
movievies = []

for movie in favorite_movies:
  print(data[data['original_title'] == movie]['vote_average'].values)
  print(data[data['original_title'] == movie]['actor_2_name'].values)
  movievies.append(data[data['original_title'] == movie]['vote_average'].values)

[7.8]
['Corey Feldman']
[7.2]
['Tsuyoshi Ihara']
[6.5]
['Charlie Day']
[7.6]
['Graham Greene']
[6.1]
['Michael Wincott']
[6.3]
['Michael Angarano']
[7.3]
['Emily Mortimer']
[6.5]
['Djimon Hounsou']
[6.6]
['Kristen Miller']
[6.3]
['Keanu Reeves']
[6.2]
['Danny Huston']
[7.3]
['Paul Giamatti']
[6.9]
['Margot Kidder']
[6.6]
['Julia Roberts']
[6.]
['Gong Li']
[]
[]
[7.3]
['Billy Crystal']
[7.4]
['Roy Scheider']
[5.5]
['Cameron Richardson']
[7.5]
['Emma Watson']
[5.9]
['Claire Danes']
[]
[]
[6.6]
['David Schwimmer']
[6.1]
['Gary Sinise']
[6.1]
['Max von Sydow']
[7.2]
['Kirsten Dunst']
[7.]
['Jack McElhone']
[6.9]
['Ernest Borgnine']
[5.7]
['Jason Schwartzman']
[]
[]
[]
[]
[6.9]
['William Zabka']
[7.5]
['Frances Fisher']
[]
[]
[5.6]
['Jason Segel']
[]
[]
[7.7]
['Hardy Krüger']
[8.]
['James Anderson']
[6.]
['Rip Torn']
[6.5]
['Billy West']
[5.2]
['John Krasinski']
[7.]
['Jude Law']
[7.2]
['Charlie Watts']
[6.1]
['Harvey Keitel']
[6.6]
['François Berléand']
[6.7]
['Cillian Murphy']
[6.7]
['Emm

## Creating Rules for our literals

In [ ]:
assertions = []

#1, 2, 3: Genre similarities
assertions.append('(find_sim_1(X, Y):- genre(X, G1),genre(Y, G1), X \= Y)')
assertions.append('(find_sim_2(X, Y):- genre(X, G1),genre(X, G2),genre(Y, G1),genre(Y, G2), G1 \= G2, X \= Y)')
assertions.append('(find_sim_3(X, Y):- genre(X, G1),genre(X, G2),genre(X, G3),genre(Y, G1),genre(Y, G2), genre(Y,G3), G1 \= G2, G2 \= G3, G3 \= G1, X \= Y)')
assertions.append('(find_sim_4(X, Y):- genre(X, G1), genre(X, G2), genre(X, G3), genre(X,G4), genre(Y, G1), genre(Y, G2), genre(Y,G3), genre(Y,G4), G1 \= G2, G2 \= G3, G3 \= G1, G1 \= G4, G2 \= G4, G3 \= G4, X \= Y)')
assertions.append('(find_sim_5(X, Y):- genre(X, G1), genre(X, G2), genre(X, G3), genre(X,G4), genre(X, G5), genre(Y, G1), genre(Y, G2), genre(Y,G3), genre(Y,G4), genre(Y, G5), G1 \= G2, G1 \= G3, G1 \= G4, G1 \= G5, G2 \= G3, G2 \= G4, G2 \= G5, G3 \= G4, G4 \= G5, X \= Y)')

#4: Same director
assertions.append('(director_sim(X, Y):- director(X, D1),director(Y, D1), X \= Y)')

#5, 6: Keyword Similarities
assertions.append('(keyword_sim_1(X, Y):- keyword(X, K1),keyword(Y, K1), X \= Y)')
assertions.append('(keyword_sim_2(X, Y):- keyword(X, K1),keyword(X, K2),keyword(Y, K1),keyword(Y, K2), K1 \= K2, X \= Y)')
assertions.append('(keyword_sim_3(X, Y):- keyword(X, K1),keyword(X, K2),keyword(X, K3),keyword(Y, K1),keyword(Y, K2), keyword(Y,K3), K1 \= K2, K2 \= K3, K3 \= K1, X \= Y)')
assertions.append('(keyword_sim_4(X, Y):- keyword(X, K1), keyword(X, K2), keyword(X, K3), keyword(X,K4), keyword(Y, K1), keyword(Y, K2), keyword(Y,K3), keyword(Y,K4), K1 \= K2, K2 \= K3, K3 \= K1, K1 \= K4, K2 \= K4, K3 \= K4, X \= Y)')
assertions.append('(keyword_sim_5(X, Y):- keyword(X, K1), keyword(X, K2), keyword(X, K3), keyword(X,K4), keyword(X, K5), keyword(Y, K1), keyword(Y, K2), keyword(Y,K3), keyword(Y,K4), keyword(Y, K5), K1 \= K2, K1 \= K3, K1 \= K4, K1 \= K5, K2 \= K3, K2 \= K4, K2 \= K5, K3 \= K4, K4 \= K5, X \= Y)')

#7, 8, 9: Same actors
assertions.append('(actor_sim_1(X, Y):- actor1(X, G1),actor1(Y, G1), X \= Y)')
assertions.append('(actor_sim_2(X, Y):- actor1(X, G1),actor2(X, G2),actor1(Y, G1),actor2(Y, G2), G1 \= G2, X \= Y)')
assertions.append('(actor_sim_3(X, Y):- actor1(X, G1),actor2(X, G2),actor3(X, G3),actor1(Y, G1),actor2(Y, G2), actor3(Y,G3), G1 \= G2, G2 \= G3, G3 \= G1, X \= Y)')

#10: Same language
assertions.append('(lang_sim(X, Y):- language(X, D1),language(Y, D1), X \= Y)')

#11: BnW
assertions.append('(bnw_sim(X, Y):- black_and_white(X, D1), black_and_white(Y, D1), X \= Y)')

#12: Same studio
assertions.append('(studio_sim_1(X, Y):- studio(X, G1),studio(Y, G1), X \= Y)')
assertions.append('(studio_sim_2(X, Y):- studio(X, G1),studio(X, G2),studio(Y, G1),studio(Y, G2), G1 \= G2, X \= Y)')
assertions.append('(studio_sim_3(X, Y):- studio(X, G1),studio(X, G2),studio(X, G3),studio(Y, G1),studio(Y, G2), studio(Y,G3), G1 \= G2, G2 \= G3, G3 \= G1, X \= Y)')
assertions.append('(studio_sim_4(X, Y):- studio(X, G1), studio(X, G2), studio(X, G3), studio(X,G4), studio(Y, G1), studio(Y, G2), studio(Y,G3), studio(Y,G4), G1 \= G2, G2 \= G3, G3 \= G1, G1 \= G4, G2 \= G4, G3 \= G4, X \= Y)')
assertions.append('(studio_sim_5(X, Y):- studio(X, G1), studio(X, G2), studio(X, G3), studio(X,G4), studio(X, G5), studio(Y, G1), studio(Y, G2), studio(Y,G3), studio(Y,G4), studio(Y, G5), G1 \= G2, G1 \= G3, G1 \= G4, G1 \= G5, G2 \= G3, G2 \= G4, G2 \= G5, G3 \= G4, G4 \= G5, X \= Y)')

#13: Same country
assertions.append('(country_sim(X, Y):- country(X, D1), country(Y, D1), X \= Y)')

#14: Same decade
assertions.append('(decade_sim(X, Y):- decade(X, D1), decade(Y, D1), X \= Y)')


In [ ]:
#assertions.append('(recommender_1(X, Y):- find_sim_1(X,Y), keyword_sim_1(X,Y))')
#assertions.append('(recommender_2(X, Y):- find_sim_1(X,Y), keyword_sim_2(X,Y))')
#assertions.append('(recommender_3(X, Y):- find_sim_1(X,Y), keyword_sim_3(X,Y))')
#assertions.append('(recommender_4(X, Y):- find_sim_1(X,Y), keyword_sim_4(X,Y))')
#assertions.append('(recommender_5(X, Y):- find_sim_1(X,Y), keyword_sim_5(X,Y))')
#assertions.append('(recommender_6(X, Y):- find_sim_2(X,Y), keyword_sim_1(X,Y))')
#assertions.append('(recommender_7(X, Y):- find_sim_2(X,Y), keyword_sim_2(X,Y))')
#assertions.append('(recommender_8(X, Y):- find_sim_2(X,Y), keyword_sim_3(X,Y))')
#assertions.append('(recommender_9(X, Y):- find_sim_2(X,Y), keyword_sim_4(X,Y))')
#assertions.append('(recommender_10(X, Y):- find_sim_2(X,Y), keyword_sim_5(X,Y))')
#assertions.append('(recommender_11(X, Y):- find_sim_3(X,Y), keyword_sim_1(X,Y))')
#assertions.append('(recommender_12(X, Y):- find_sim_3(X,Y), keyword_sim_2(X,Y))')
#assertions.append('(recommender_13(X, Y):- find_sim_3(X,Y), keyword_sim_3(X,Y))')
#assertions.append('(recommender_14(X, Y):- find_sim_3(X,Y), keyword_sim_4(X,Y))')
#assertions.append('(recommender_15(X, Y):- find_sim_3(X,Y), keyword_sim_5(X,Y))')

In [ ]:
#assertions.append('(recommender_1(X, Y):- find_sim_2(X,Y), keyword_sim_1(X,Y),score(Y,S), atom_number(S,Z), Z > 6)')
#assertions.append('(recommender_2(X, Y):- find_sim_2(X,Y), keyword_sim_2(X,Y),score(Y,S), atom_number(S,Z), Z > 6)')
#assertions.append('(recommender_3(X, Y):- find_sim_2(X,Y), keyword_sim_3(X,Y),score(Y,S), atom_number(S,Z), Z > 6)')
#assertions.append('(recommender_4(X, Y):- find_sim_2(X,Y), keyword_sim_4(X,Y),score(Y,S), atom_number(S,Z), Z > 6)')
#assertions.append('(recommender_5(X, Y):- find_sim_2(X,Y), keyword_sim_5(X,Y),score(Y,S), atom_number(S,Z), Z > 6)')
#assertions.append('(recommender_6(X, Y):- find_sim_2(X,Y), keyword_sim_1(X,Y),score(Y,S), atom_number(S,Z), Z > 6)')
#assertions.append('(recommender_7(X, Y):- find_sim_2(X,Y), keyword_sim_2(X,Y),score(Y,S), atom_number(S,Z), Z > 6)')
#assertions.append('(recommender_8(X, Y):- find_sim_2(X,Y), keyword_sim_3(X,Y),score(Y,S), atom_number(S,Z), Z > 6)')
#assertions.append('(recommender_9(X, Y):- find_sim_2(X,Y), keyword_sim_4(X,Y),score(Y,S), atom_number(S,Z), Z > 6)')
#assertions.append('(recommender_10(X, Y):- find_sim_2(X,Y), keyword_sim_5(X,Y),score(Y,S), atom_number(S,Z), Z > 6)')
#assertions.append('(recommender_11(X, Y):- find_sim_3(X,Y), keyword_sim_1(X,Y),score(Y,S), atom_number(S,Z), Z > 6)')
#assertions.append('(recommender_12(X, Y):- find_sim_3(X,Y), keyword_sim_2(X,Y),score(Y,S), atom_number(S,Z), Z > 6)')
#assertions.append('(recommender_13(X, Y):- find_sim_3(X,Y), keyword_sim_3(X,Y),score(Y,S), atom_number(S,Z), Z > 6)')
#assertions.append('(recommender_14(X, Y):- find_sim_3(X,Y), keyword_sim_4(X,Y),score(Y,S), atom_number(S,Z), Z > 6)')
#assertions.append('(recommender_15(X, Y):- find_sim_3(X,Y), keyword_sim_5(X,Y),score(Y,S), atom_number(S,Z), Z > 6)')


In [ ]:
#Improved/Final Recommender Functions
assertions.append('(recommender_1(X, Y):- find_sim_1(X,Y), actor_sim_1(X,Y))')
assertions.append('(recommender_2(X, Y):- find_sim_1(X,Y), keyword_sim_1(X,Y))')
assertions.append('(recommender_3(X, Y):- find_sim_1(X,Y), (actor_sim_1(X,Y) ; keyword_sim_1(X,Y)))')
assertions.append('(recommender_4(X, Y):- find_sim_1(X,Y), (keyword_sim_1(X,Y) ; (score(Y,S), atom_number(S,Z), Z > 6)))')
assertions.append('(recommender_5(X, Y):- find_sim_2(X,Y), (decade_sim(X,Y) ; country_sim(X,Y)))')

for assertion in assertions:
  prolog.assertz(assertion)

## Creating our Recommender Function

In [ ]:
def my_recommender(movie):
    s = set()
    list_of_numbers = [5,4,3,2,1]
    threshold = []
    index = 0
    for i in list_of_numbers:
      #print(f"recommender {i} has begun running.")
      q = prolog.query(f"recommender_{i}('" + movie +"',M)")
      for soln in q:
        m = soln['M']
        if m not in s:
            index += 1
            s.add(soln['M'])
      #print(f"recommender {i} has capped at {index}")
      threshold.append(index)
      #print(f"threshold is {threshold}")
      q.close()
    print(f"final threshold is {threshold}")
    answers = s
    return answers, threshold

In [ ]:
with open(path + "literals.pl", "w") as f:
    for l in literals:
      f.write(f"{l}\n")

In [ ]:
with open(path + "assertions.pl", "w") as f:
    for a in assertions:
      f.write(f"{a}\n")

In [ ]:
# prolog.assertz('(directed_by(X,Y) :- findall(M,director(M,X),Y))')

In [ ]:
data.columns

Index(['Unnamed: 0', 'budget', 'genres', 'homepage', 'id', 'plot_keywords',
       'language', 'original_title', 'overview', 'popularity',
       'production_companies', 'production_countries', 'release_date', 'gross',
       'duration', 'spoken_languages', 'status', 'tagline', 'movie_title',
       'vote_average', 'num_voted_users', 'title_year', 'country',
       'director_name', 'actor_1_name', 'actor_2_name', 'actor_3_name'],
      dtype='object')

In [ ]:
data[data['plot_keywords'].str.contains('violence')]

,Unnamed: 0,budget,genres,homepage,id,plot_keywords,language,original_title,overview,popularity,...,tagline,movie_title,vote_average,num_voted_users,title_year,country,director_name,actor_1_name,actor_2_name,actor_3_name
176,176,135000000,Western|Drama|Adventure|Thriller,http://www.foxmovies.com/movies/the-revenant,281957,father son relationship|rape|based on novel|mo...,English,The Revenant,"In the 1820s, a frontiersman, Hugh Glass, sets...",100.635882,...,"(n. One who has returned, as if from the dead.)",The Revenant,7.3,6396,2015.0,Canada,Alejandro González Iñárritu,Tom Hardy,Will Poulter,Domhnall Gleeson
180,180,70000000,Action|Drama|Mystery|Thriller,http://www.universalstudiosentertainment.com/t...,2503,paris|corruption|madrid|assassin|based on nove...,English,The Bourne Ultimatum,Bourne is brought out of hiding once again by ...,45.381501,...,Remember everything. Forgive nothing.,The Bourne Ultimatum,7.3,2888,2007.0,Germany,Paul Greengrass,Julia Stiles,David Strathairn,Scott Glenn
186,186,130000000,Adventure|Action|Comedy|Thriller|Crime,UNK,8961,miami|ku klux klan|cuba|undercover|mexican sta...,English,Bad Boys II,"Out-of-control, trash-talking buddy cops Marcu...",38.068736,...,"If you can't stand the heat, get out of Miami.",Bad Boys II,6.3,1564,2003.0,United States of America,Michael Bay,Will Smith,Jordi Mollà,Gabrielle Union
188,188,110000000,Action|Mystery|Thriller,http://www.sonypictures.com/homevideo/salt/,27576,assassination|spy|cia|kidnapping|cold war|sovi...,English,Salt,"As a CIA officer, Evelyn Salt swore an oath to...",48.829437,...,Who is Salt?,Salt,6.2,2093,2010.0,United States of America,Phillip Noyce,Liev Schreiber,Chiwetel Ejiofor,Daniel Olbrychski
213,213,125000000,Adventure|Action|Thriller,http://www.missionimpossible.com/,955,terror|spain|cia|helicopter|secret identity|sk...,English,Mission: Impossible II,With computer genius Luther Stickell at his si...,54.931334,...,Expect the impossible again.,Mission: Impossible II,5.9,1928,2000.0,Germany,John Woo,Dougray Scott,Thandie Newton,Ving Rhames
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4545,4545,500000,Comedy|Horror,http://www.detentionofthedead.com,139715,high school|independent film|zombie|extreme vi...,English,Detention of the Dead,A group of oddball high school students find t...,2.391847,...,"When there's no more room in Hell, the dead wi...",Detention of the Dead,4.5,34,2012.0,United States of America,Alex Craig Mann,Alexa Nikolas,Christa Brittany Allen,Jayson Blair
4585,4585,0,Horror,UNK,19204,hotel|experiment|hell|gore|morgue|undead|blood...,English,...E tu vivrai nel terrore! L'aldilà,A young woman inherits an old hotel in Louisia...,8.022122,...,The seven dreaded gateways to Hell are conceal...,The Beyond,6.6,117,1981.0,Italy,Lucio Fulci,David Warbeck,Cinzia Monreale,Antoine Saint-John
4639,4639,250000,Thriller|Science Fiction|Mystery,UNK,431,riddle|autism|claustrophobia|maze|prime number...,Deutsch,Cube,Seven strangers are taken out of their daily l...,44.656151,...,Don't look for a reason... Look for a way out.,Cube,6.9,1069,1997.0,Canada,Vincenzo Natali,Nicky Guadagni,David Hewlett,Andrew Miller
4732,4732,0,Thriller,UNK,13538,london england|countryside|rape|sex|trauma|van...,English,Straightheads,There is instant chemistry between Alice (Gill...,8.302951,...,Let the revenge fit the crime,Straightheads,5.2,40,2007.0,United Kingdom,Dan Reed,Danny Dyer,Adam Rayner,Antony Byrne


In [ ]:
def simple_recommender(movie, number):
    s = set()
    list_of_numbers = [1, 2, 3, 4, 5]
    threshold = []
    index = 0
    for i in list_of_numbers:
      print(f"recommender {i} has begun running.")
      q = prolog.query(v)
      for soln in q:
        m = soln['M']
        if m not in s:
            index += 1
            s.add(soln['M'])
    threshold.append(index)
    q.close()
    answers = s
    return s, threshold

In [ ]:
movies, _ = my_recommender('Avatar')

final threshold is [622, 1339, 1339, 1339, 1339]


In [ ]:
list(movies)

['Big Hero 6',
 'Edge of Tomorrow',
 'Force 10 from Navarone',
 'Gridiron Gang',
 'Catch a Fire',
 'Harley Davidson and the Marlboro Man',
 'How to Train Your Dragon',
 'Fantastic Four',
 'Southland Tales',
 'Titan A.E.',
 'The Interview',
 'Jaws',
 'Megaforce',
 'Silver Medalist',
 'Creepshow',
 'The Imaginarium of Doctor Parnassus',
 'Barbarella',
 'Vanilla Sky',
 'Gremlins 2: The New Batch',
 'The Losers',
 'Ice Age: Dawn of the Dinosaurs',
 'Riddick',
 'WarGames',
 'Rush Hour 3',
 'Poseidon',
 'Taken',
 'What Dreams May Come',
 'Highlander',
 'U-571',
 'Percy Jackson: Sea of Monsters',
 'The Last Dragon',
 'Steel',
 'Rush Hour 2',
 'Rabbit-Proof Fence',
 'Troy',
 'The Wild Bunch',
 'Taken 3',
 'The Art of War',
 'The Running Man',
 'Dracula Untold',
 'Pirates of the Caribbean: At Worlds End',
 'The Transporter',
 'Pacific Rim',
 'Kick-Ass',
 'The Prophecy',
 'Bubba Ho-tep',
 'Rambo: First Blood Part II',
 'Pandorum',
 'Payback',
 '10th & Wolf',
 'Prometheus',
 'Midnight Special',
 

## Checking how our recommender did 

In [ ]:
from tqdm.notebook import tqdm
from sklearn.metrics import precision_score, recall_score, f1_score
import numpy as np
import random

rating_weights = {0: -1, 1: -0.5, 2:0, 3:0, 4:0.5, 5:1}
score_weights = {i:100/(i+1) for i in range(7)}

def train_recommender(ratings, rating_weights, score_weights, number_of_movies = 10):

    if number_of_movies > len(ratings):
        number_of_movies = len(ratings)


    if number_of_movies != -1:
        indexes = random.sample(range(len(ratings)), number_of_movies)
        ratings = ratings.iloc[indexes]

    movie_score = {}
    for row in tqdm(ratings.itertuples(index=True, name='Pandas')):
        movie = clean_text(getattr(row, 'movie_title'))
        #print(f"Finding similar movies to {movie}")
        rating = getattr(row, 'rating')



        similar_movies, threshold = my_recommender(movie)
        current_index = 0

        try:
          while threshold[current_index] == 0:
              current_index += 1
        except:
          continue


        for i,similar_movie in enumerate(similar_movies):


            if i+1 > threshold[current_index]:
              current_index += 1

            if similar_movie not in movie_score:
                movie_score[similar_movie] = rating_weights[int(rating)] * score_weights[current_index]
            else:
                movie_score[similar_movie] += rating_weights[int(rating)] * score_weights[current_index] # το weight θα το ορίσετε ανα επίπεδο ομοιότητας οι πολύ όμοιες ταινίες θα έχουν μεγαλύτερο βάρος



    return movie_score


def predict_example(ratings, movie_score):
    real, pred = [], []
    for i, row in enumerate(ratings.itertuples(index=True, name='Pandas')):
        movie = clean_text(getattr(row, 'movie_title'))
        rating = getattr(row, 'rating')

        if movie in movie_score:
            pred.append(int(movie_score[movie] > 0)) 
            real.append(int(rating > 3))
        else: 
            pred.append(0)
            real.append(int(rating > 3))

    return real, pred


def get_metrics(real, pred):
    metrics = {}
    metrics["precision"] = precision_score(real, pred)
    metrics["recall"] = recall_score(real, pred)
    metrics["f1"] = f1_score(real, pred)
    return metrics

In [ ]:
train_ratings = pd.read_csv(path + "train_ratings.csv")
test_ratings = pd.read_csv(path + "test_ratings.csv")

In [ ]:
metrics = []
for i in range (10):
    movie_score = train_recommender(train_ratings, rating_weights, score_weights, 10)
    real, pred = predict_example(test_ratings, movie_score)
    metrics.append(get_metrics(real, pred))

for metric in metrics[0].keys():
    print (f"{metric}: {np.mean([m[metric] for m in metrics])}")

0it [00:00, ?it/s]

final threshold is [1011, 2550, 2551, 2551, 2551]
final threshold is [187, 869, 869, 869, 869]
final threshold is [501, 1180, 1180, 1180, 1180]
final threshold is [96, 912, 912, 912, 912]
final threshold is [178, 1310, 1310, 1310, 1310]
final threshold is [671, 2354, 2356, 2356, 2356]
final threshold is [0, 0, 0, 0, 0]
final threshold is [418, 2246, 2248, 2248, 2248]
final threshold is [472, 1368, 1368, 1368, 1368]
final threshold is [438, 1394, 1397, 1397, 1397]


0it [00:00, ?it/s]

final threshold is [8, 1983, 1983, 1983, 1983]
final threshold is [287, 1861, 1861, 1861, 1861]
final threshold is [95, 1737, 1737, 1737, 1737]
final threshold is [49, 1972, 1972, 1972, 1972]
final threshold is [692, 2421, 2422, 2422, 2422]
final threshold is [100, 1070, 1070, 1070, 1070]
final threshold is [825, 2583, 2583, 2583, 2583]
final threshold is [1011, 2479, 2479, 2479, 2479]
final threshold is [24, 1638, 1638, 1638, 1638]
final threshold is [1773, 3068, 3069, 3069, 3069]


0it [00:00, ?it/s]

...

final threshold is [470, 2158, 2158, 2158, 2158]
final threshold is [61, 1939, 1939, 1939, 1939]
final threshold is [185, 782, 782, 782, 782]
final threshold is [0, 1597, 1597, 1597, 1597]
final threshold is [501, 1180, 1180, 1180, 1180]
final threshold is [0, 878, 879, 879, 879]
final threshold is [157, 1112, 1112, 1112, 1112]
final threshold is [91, 928, 928, 928, 928]
final threshold is [0, 307, 307, 307, 307]
final threshold is [271, 702, 702, 702, 702]


0it [00:00, ?it/s]

final threshold is [252, 1039, 1039, 1039, 1039]
final threshold is [316, 2233, 2234, 2234, 2234]
final threshold is [10, 736, 736, 736, 736]
final threshold is [91, 928, 928, 928, 928]
final threshold is [634, 2221, 2221, 2221, 2221]
final threshold is [326, 1883, 1883, 1883, 1883]
final threshold is [300, 1253, 1253, 1253, 1253]
final threshold is [863, 2524, 2524, 2524, 2524]
final threshold is [0, 843, 845, 845, 845]
final threshold is [411, 1791, 1791, 1791, 1791]


0it [00:00, ?it/s]

final threshold is [512, 2583, 2583, 2583, 2583]
final threshold is [288, 951, 951, 951, 951]
final threshold is [236, 1868, 1868, 1868, 1868]
final threshold is [418, 2246, 2248, 2248, 2248]
final threshold is [501, 1180, 1180, 1180, 1180]
final threshold is [0, 1597, 1597, 1597, 1597]
final threshold is [23, 2073, 2073, 2073, 2073]
final threshold is [490, 2243, 2244, 2244, 2244]
final threshold is [600, 2287, 2287, 2287, 2287]
final threshold is [62, 1927, 1927, 1927, 1927]
precision: 0.5271437689359116
recall: 0.5777777777777777
f1: 0.5234214033534526


In [ ]:
metrics = []
for i in range (10):
    movie_score = train_recommender(train_ratings, rating_weights, score_weights, 30)
    real, pred = predict_example(test_ratings, movie_score)
    metrics.append(get_metrics(real, pred))

for metric in metrics[0].keys():
    print (f"{metric}: {np.mean([m[metric] for m in metrics])}")

0it [00:00, ?it/s]

final threshold is [326, 1883, 1883, 1883, 1883]
final threshold is [0, 1597, 1597, 1597, 1597]
final threshold is [805, 1614, 1614, 1614, 1614]
final threshold is [185, 782, 782, 782, 782]
final threshold is [747, 2337, 2337, 2337, 2337]
final threshold is [178, 1310, 1310, 1310, 1310]
final threshold is [49, 1972, 1972, 1972, 1972]
final threshold is [0, 0, 0, 0, 0]
final threshold is [669, 2487, 2488, 2488, 2488]
final threshold is [1011, 2479, 2479, 2479, 2479]
final threshold is [547, 1472, 1472, 1472, 1472]
final threshold is [0, 307, 307, 307, 307]
final threshold is [288, 951, 951, 951, 951]
final threshold is [236, 1864, 1864, 1864, 1864]
final threshold is [113, 1798, 1798, 1798, 1798]
final threshold is [520, 1469, 1469, 1469, 1469]
final threshold is [287, 1861, 1861, 1861, 1861]
final threshold is [240, 922, 922, 922, 922]
final threshold is [113, 1639, 1639, 1639, 1639]
final threshold is [470, 2158, 2158, 2158, 2158]
final threshold is [96, 912, 912, 912, 912]
final thre

0it [00:00, ?it/s]

final threshold is [95, 1737, 1737, 1737, 1737]
final threshold is [236, 1868, 1868, 1868, 1868]
final threshold is [0, 866, 866, 866, 866]
final threshold is [326, 1883, 1883, 1883, 1883]
final threshold is [100, 1070, 1070, 1070, 1070]
final threshold is [0, 878, 879, 879, 879]
final threshold is [1735, 3233, 3234, 3234, 3234]
final threshold is [418, 2246, 2248, 2248, 2248]
final threshold is [271, 702, 702, 702, 702]
final threshold is [553, 2728, 2728, 2728, 2728]
final threshold is [113, 1639, 1639, 1639, 1639]
final threshold is [10, 736, 736, 736, 736]
final threshold is [671, 2354, 2356, 2356, 2356]
final threshold is [0, 952, 954, 954, 954]
final threshold is [406, 1853, 1853, 1853, 1853]
final threshold is [1011, 2550, 2551, 2551, 2551]
final threshold is [1011, 2568, 2568, 2568, 2568]
final threshold is [96, 912, 912, 912, 912]
final threshold is [505, 2269, 2269, 2269, 2269]
final threshold is [0, 307, 307, 307, 307]
final threshold is [130, 1341, 1341, 1341, 1341]
final t

0it [00:00, ?it/s]

...

final threshold is [931, 1746, 1746, 1746, 1746]
final threshold is [271, 702, 702, 702, 702]
final threshold is [634, 2221, 2221, 2221, 2221]
final threshold is [58, 2060, 2060, 2060, 2060]
final threshold is [863, 2524, 2524, 2524, 2524]
final threshold is [668, 1547, 1547, 1547, 1547]
final threshold is [411, 1791, 1791, 1791, 1791]
final threshold is [604, 1945, 1945, 1945, 1945]
final threshold is [1773, 3068, 3069, 3069, 3069]
final threshold is [572, 2406, 2409, 2409, 2409]
final threshold is [236, 1837, 1840, 1840, 1840]
final threshold is [91, 928, 928, 928, 928]
final threshold is [825, 2583, 2583, 2583, 2583]
final threshold is [0, 1595, 1595, 1595, 1595]
final threshold is [0, 843, 845, 845, 845]
final threshold is [178, 1310, 1310, 1310, 1310]
final threshold is [418, 2307, 2307, 2307, 2307]
final threshold is [0, 858, 858, 858, 858]
final threshold is [324, 1914, 1915, 1915, 1915]
final threshold is [96, 912, 912, 912, 912]
final threshold is [444, 2157, 2158, 2158, 2158]

0it [00:00, ?it/s]

final threshold is [1735, 3233, 3234, 3234, 3234]
final threshold is [167, 348, 348, 348, 348]
final threshold is [252, 1039, 1039, 1039, 1039]
final threshold is [23, 2073, 2073, 2073, 2073]
final threshold is [418, 2307, 2307, 2307, 2307]
final threshold is [1011, 2568, 2568, 2568, 2568]
final threshold is [931, 1746, 1746, 1746, 1746]
final threshold is [406, 1853, 1853, 1853, 1853]
final threshold is [634, 2221, 2221, 2221, 2221]
final threshold is [0, 858, 858, 858, 858]
final threshold is [1011, 2479, 2479, 2479, 2479]
final threshold is [113, 1798, 1798, 1798, 1798]
final threshold is [668, 1547, 1547, 1547, 1547]
final threshold is [256, 1285, 1285, 1285, 1285]
final threshold is [505, 2269, 2269, 2269, 2269]
final threshold is [604, 1945, 1945, 1945, 1945]
final threshold is [0, 228, 228, 228, 228]
final threshold is [418, 2246, 2248, 2248, 2248]
final threshold is [435, 2157, 2157, 2157, 2157]
final threshold is [553, 1484, 1484, 1484, 1484]
final threshold is [1773, 3068, 30

0it [00:00, ?it/s]

final threshold is [863, 2524, 2524, 2524, 2524]
final threshold is [326, 1883, 1883, 1883, 1883]
final threshold is [91, 928, 928, 928, 928]
final threshold is [553, 2728, 2728, 2728, 2728]
final threshold is [209, 1277, 1278, 1278, 1278]
final threshold is [512, 2583, 2583, 2583, 2583]
final threshold is [367, 1354, 1354, 1354, 1354]
final threshold is [113, 1639, 1639, 1639, 1639]
final threshold is [352, 1159, 1159, 1159, 1159]
final threshold is [572, 2406, 2409, 2409, 2409]
final threshold is [61, 1939, 1939, 1939, 1939]
final threshold is [81, 1272, 1272, 1272, 1272]
final threshold is [0, 241, 241, 241, 241]
final threshold is [300, 1253, 1253, 1253, 1253]
final threshold is [825, 2583, 2583, 2583, 2583]
final threshold is [324, 1914, 1915, 1915, 1915]
final threshold is [24, 1638, 1638, 1638, 1638]
final threshold is [113, 1798, 1798, 1798, 1798]
final threshold is [236, 1868, 1868, 1868, 1868]
final threshold is [90, 1735, 1736, 1736, 1736]
final threshold is [240, 922, 922, 

In [ ]:
metrics = []
for i in range (10):
    movie_score = train_recommender(train_ratings, rating_weights, score_weights, -1)
    real, pred = predict_example(test_ratings, movie_score)
    metrics.append(get_metrics(real, pred))

for metric in metrics[0].keys():
    print (f"{metric}: {np.mean([m[metric] for m in metrics])}")

0it [00:00, ?it/s]

final threshold is [236, 1864, 1864, 1864, 1864]
final threshold is [348, 2282, 2282, 2282, 2282]
final threshold is [287, 1861, 1861, 1861, 1861]
final threshold is [326, 1883, 1883, 1883, 1883]
final threshold is [130, 1341, 1341, 1341, 1341]
final threshold is [49, 1972, 1972, 1972, 1972]
final threshold is [472, 1368, 1368, 1368, 1368]
final threshold is [470, 2158, 2158, 2158, 2158]
final threshold is [252, 1039, 1039, 1039, 1039]
final threshold is [316, 2233, 2234, 2234, 2234]
final threshold is [501, 1180, 1180, 1180, 1180]
final threshold is [157, 1112, 1112, 1112, 1112]
final threshold is [671, 2354, 2356, 2356, 2356]
final threshold is [805, 1614, 1614, 1614, 1614]
final threshold is [411, 1791, 1791, 1791, 1791]
final threshold is [0, 241, 241, 241, 241]
final threshold is [1011, 2550, 2551, 2551, 2551]
final threshold is [0, 1597, 1597, 1597, 1597]
final threshold is [0, 0, 0, 0, 0]
final threshold is [604, 1945, 1945, 1945, 1945]
final threshold is [209, 1277, 1278, 1278,

0it [00:00, ?it/s]

final threshold is [236, 1864, 1864, 1864, 1864]
final threshold is [348, 2282, 2282, 2282, 2282]
final threshold is [287, 1861, 1861, 1861, 1861]
final threshold is [326, 1883, 1883, 1883, 1883]
final threshold is [130, 1341, 1341, 1341, 1341]
final threshold is [49, 1972, 1972, 1972, 1972]
final threshold is [472, 1368, 1368, 1368, 1368]
final threshold is [470, 2158, 2158, 2158, 2158]
final threshold is [252, 1039, 1039, 1039, 1039]
final threshold is [316, 2233, 2234, 2234, 2234]
final threshold is [501, 1180, 1180, 1180, 1180]
final threshold is [157, 1112, 1112, 1112, 1112]
final threshold is [671, 2354, 2356, 2356, 2356]
final threshold is [805, 1614, 1614, 1614, 1614]
final threshold is [411, 1791, 1791, 1791, 1791]
final threshold is [0, 241, 241, 241, 241]
final threshold is [1011, 2550, 2551, 2551, 2551]
final threshold is [0, 1597, 1597, 1597, 1597]
final threshold is [0, 0, 0, 0, 0]
final threshold is [604, 1945, 1945, 1945, 1945]
final threshold is [209, 1277, 1278, 1278,

0it [00:00, ?it/s]

...

final threshold is [236, 1864, 1864, 1864, 1864]
final threshold is [348, 2282, 2282, 2282, 2282]
final threshold is [287, 1861, 1861, 1861, 1861]
final threshold is [326, 1883, 1883, 1883, 1883]
final threshold is [130, 1341, 1341, 1341, 1341]
final threshold is [49, 1972, 1972, 1972, 1972]
final threshold is [472, 1368, 1368, 1368, 1368]
final threshold is [470, 2158, 2158, 2158, 2158]
final threshold is [252, 1039, 1039, 1039, 1039]
final threshold is [316, 2233, 2234, 2234, 2234]
final threshold is [501, 1180, 1180, 1180, 1180]
final threshold is [157, 1112, 1112, 1112, 1112]
final threshold is [671, 2354, 2356, 2356, 2356]
final threshold is [805, 1614, 1614, 1614, 1614]
final threshold is [411, 1791, 1791, 1791, 1791]
final threshold is [0, 241, 241, 241, 241]
final threshold is [1011, 2550, 2551, 2551, 2551]
final threshold is [0, 1597, 1597, 1597, 1597]
final threshold is [0, 0, 0, 0, 0]
final threshold is [604, 1945, 1945, 1945, 1945]
final threshold is [209, 1277, 1278, 1278,

0it [00:00, ?it/s]

final threshold is [236, 1864, 1864, 1864, 1864]
final threshold is [348, 2282, 2282, 2282, 2282]
final threshold is [287, 1861, 1861, 1861, 1861]
final threshold is [326, 1883, 1883, 1883, 1883]
final threshold is [130, 1341, 1341, 1341, 1341]
final threshold is [49, 1972, 1972, 1972, 1972]
final threshold is [472, 1368, 1368, 1368, 1368]
final threshold is [470, 2158, 2158, 2158, 2158]
final threshold is [252, 1039, 1039, 1039, 1039]
final threshold is [316, 2233, 2234, 2234, 2234]
final threshold is [501, 1180, 1180, 1180, 1180]
final threshold is [157, 1112, 1112, 1112, 1112]
final threshold is [671, 2354, 2356, 2356, 2356]
final threshold is [805, 1614, 1614, 1614, 1614]
final threshold is [411, 1791, 1791, 1791, 1791]
final threshold is [0, 241, 241, 241, 241]
final threshold is [1011, 2550, 2551, 2551, 2551]
final threshold is [0, 1597, 1597, 1597, 1597]
final threshold is [0, 0, 0, 0, 0]
final threshold is [604, 1945, 1945, 1945, 1945]
final threshold is [209, 1277, 1278, 1278,

0it [00:00, ?it/s]

final threshold is [236, 1864, 1864, 1864, 1864]
final threshold is [348, 2282, 2282, 2282, 2282]
final threshold is [287, 1861, 1861, 1861, 1861]
final threshold is [326, 1883, 1883, 1883, 1883]
final threshold is [130, 1341, 1341, 1341, 1341]
final threshold is [49, 1972, 1972, 1972, 1972]
final threshold is [472, 1368, 1368, 1368, 1368]
final threshold is [470, 2158, 2158, 2158, 2158]
final threshold is [252, 1039, 1039, 1039, 1039]
final threshold is [316, 2233, 2234, 2234, 2234]
final threshold is [501, 1180, 1180, 1180, 1180]
final threshold is [157, 1112, 1112, 1112, 1112]
final threshold is [671, 2354, 2356, 2356, 2356]
final threshold is [805, 1614, 1614, 1614, 1614]
final threshold is [411, 1791, 1791, 1791, 1791]
final threshold is [0, 241, 241, 241, 241]
final threshold is [1011, 2550, 2551, 2551, 2551]
final threshold is [0, 1597, 1597, 1597, 1597]
final threshold is [0, 0, 0, 0, 0]
final threshold is [604, 1945, 1945, 1945, 1945]
final threshold is [209, 1277, 1278, 1278,